<a href="https://colab.research.google.com/github/Soham-Bundela/College_Work_Artificial_Intelligence/blob/main/AO_Algorithm_for_Problem_Reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math

def ao_star(graph, heuristic, start_node):
    """
    Implements a simplified AO* search algorithm for And/Or graphs.

    The AO* algorithm iteratively finds the best partial solution graph
    by expanding the most promising unexpanded node and propagating cost updates.

    :param graph: A dictionary representing the And/Or graph.
        - Keys are nodes (problems).
        - Values are lists of successor groups (possible solution methods).
        - Successor Group Format: (cost_to_reach_successors, is_and_node, list_of_successors)
          - is_and_node=True means all successors must be solved (AND relationship).
          - is_and_node=False means only one successor must be solved (OR relationship).
    :param heuristic: A dictionary mapping nodes to their estimated cost (h-value).
    :param start_node: The initial problem node.
    :return: The final heuristic costs and the best solution graph found.
    """

    # 1. Initialization
    # h: The current estimated cost (h-value) for each node.
    h = heuristic.copy()

    # solution_graph: The current best partial solution structure.
    # solution_graph[node] = (cost_to_reach_successors, is_and_node, list_of_successors)
    solution_graph = {}

    # List of nodes that need cost re-evaluation (starts with the start node)
    # The start node is always the current focus until its cost stabilizes.
    open_list = [start_node]

    print(f"--- Starting AO* Search from {start_node} ---")

    while open_list:
        # Get the current node to process (LIFO is often used for depth-first approach)
        current_node = open_list.pop()

        # Check if the node is already solved (h[node] == 0 means a terminal/solved node)
        if h[current_node] == 0:
            print(f"Node {current_node} is a terminal state (h=0). Skipping expansion.")
            continue

        print(f"\nEvaluating Node: {current_node} (Current h={h[current_node]})")

        # 2. Expansion: Find the best successors for the current node
        best_successor_group = None
        min_cost = float('inf')

        # graph[current_node] is a list of OR options
        for group in graph.get(current_node, []):
            cost_to_successors, is_and, successors = group

            # Calculate the total estimated cost for this option (group)
            # Total Cost = Cost to reach successors + Sum of successor h-values
            successor_h_sum = sum(h.get(s, math.inf) for s in successors)
            group_cost = cost_to_successors + successor_h_sum

            print(f"  Option: {successors} (AND={is_and}), Cost: {group_cost:.2f}")

            # Select the option with the minimum total cost
            if group_cost < min_cost:
                min_cost = group_cost
                best_successor_group = group

        # 3. Update Current Node's Cost and Best Path
        if best_successor_group:
            # Update the cost of the current node to the cost of the best option
            new_h = min_cost
            print(f"  -> Best Option Selected. New h[{current_node}] = {new_h:.2f}")

            # Only proceed if a cost change occurred
            if new_h < h[current_node]:
                # Update the cost
                h[current_node] = new_h
                # Record the best path (the chosen successor group)
                solution_graph[current_node] = best_successor_group

                # 4. Propagation (Backtracking): Add parents to re-evaluate
                # In a full AO* implementation, you would need to track parents and
                # recursively update all ancestors. For this short code, we rely
                # on re-processing the nodes that have neighbors in the solution graph.

                # Add the neighbors of the chosen best path to the open list for future expansion
                # Only add if the neighbor is not yet solved (h > 0)
                for successor in best_successor_group[2]:
                    if h.get(successor, math.inf) > 0 and successor not in open_list:
                        open_list.append(successor)

        else:
            # If there are no options, the problem is unsolvable from here.
            h[current_node] = math.inf


    return h[start_node], solution_graph

# --- Example Usage ---
if __name__ == "__main__":

    # Define the graph structure (A, B, C, D are problems; Z is the solved terminal state)
    # The graph represents the problem 'A' which can be solved in two ways:
    # 1. OR: Transform 'A' into problem 'B' (cost 1).
    # 2. AND: Transform 'A' into problems 'C' AND 'D' (cost 2).
    graph = {
        # Format: (cost_to_successors, is_and_node, [successors])
        'A': [
            (1, False, ['B']),      # OR Option 1: A -> B (cost 1)
            (2, True, ['C', 'D'])   # AND Option 2: A -> C AND D (cost 2)
        ],
        'B': [
            (3, False, ['Z'])       # OR Option: B -> Z (cost 3)
        ],
        'C': [
            (1, False, ['Z'])       # OR Option: C -> Z (cost 1)
        ],
        'D': [
            (1, False, ['Z'])       # OR Option: D -> Z (cost 1)
        ],
        'Z': [] # Terminal state, no successors
    }

    # Initial Heuristic (h-value): Estimated cost to solve the problem
    heuristic = {
        'A': 10,  # A is initially expensive
        'B': 5,
        'C': 5,
        'D': 5,
        'Z': 0    # Z is solved, so cost is 0
    }

    start = 'A'

    final_cost, solution = ao_star(graph, heuristic, start)

    print("\n--- AO* Results ---")
    print(f"Start Node: {start}")
    print(f"Final Estimated Solution Cost: {final_cost:.2f}")
    print("\nBest Solution Graph:")

    # Print the resulting optimal plan
    def print_solution(node, indent=0):
        if node not in solution:
            print("  " * indent + f"- {node} (Solved)")
            return

        cost_to_successors, is_and, successors = solution[node]

        successor_type = "AND" if is_and else "OR"
        print("  " * indent + f"-> Solve {node} by {successor_type} solving:")

        for succ in successors:
            print_solution(succ, indent + 1)

    print_solution(start)

--- Starting AO* Search from A ---

Evaluating Node: A (Current h=10)
  Option: ['B'] (AND=False), Cost: 6.00
  Option: ['C', 'D'] (AND=True), Cost: 12.00
  -> Best Option Selected. New h[A] = 6.00

Evaluating Node: B (Current h=5)
  Option: ['Z'] (AND=False), Cost: 3.00
  -> Best Option Selected. New h[B] = 3.00

--- AO* Results ---
Start Node: A
Final Estimated Solution Cost: 6.00

Best Solution Graph:
-> Solve A by OR solving:
  -> Solve B by OR solving:
    - Z (Solved)
